# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
import re
def get_list_of_university_towns():
    uni_tw = open('university_towns.txt', 'r')
    lines = uni_tw.readlines()
    ls = ''
#     for line in uni_tw:
#         ls += line
#     print(type(ls))
    i = 0
    data_dic = {}
    state = ''
    city = ''
    df_res = pd.DataFrame(columns=('State', 'RegionName'))
    for each in lines:
#         each = each.rstrip()
        each = each.rstrip()
        if (re.findall('\[edit\]', each)):
            state = re.sub('\[edit\]|', '', each)
        else:
            city = re.sub(' \(.*', '', each)
            df_res.loc[i] = [state, city]
#         lines[i] = each
        i += 1
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return df_res
get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows=220, header=None)
#     print(GDP)
    GDP = GDP[[4, 5, 6]]
    GDP = pd.DataFrame(GDP)
    GDP.rename(columns={4: 'date', 5 : 'GDP in billions of current dollars', 6 : 'GDP in billions of chained 2009 dollars'}, inplace=True)
    GDP.set_index('date', inplace=True)
#     print(len(GDP['GDP in billions of chained 2009 dollars']))
    for i in range(len(GDP['GDP in billions of chained 2009 dollars'])):
        if GDP['GDP in billions of chained 2009 dollars'][i] > GDP['GDP in billions of chained 2009 dollars'][i+1]:
                if GDP['GDP in billions of chained 2009 dollars'][i + 1] > GDP['GDP in billions of chained 2009 dollars'][i+2]:
                    return GDP.iloc[i + 1].name
# GDP in billions of current dollars 5
# GDP in billions of chained 2009 dollars 6

get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows=220, header=None)
#     print(GDP)
    j = 0
    GDP = GDP[[4, 5, 6]]
    GDP = pd.DataFrame(GDP)
    GDP.rename(columns={4: 'date', 5 : 'GDP in billions of current dollars', 6 : 'GDP in billions of chained 2009 dollars'}, inplace=True)
    GDP.set_index('date', inplace=True)
#     print(GDP['GDP in billions of chained 2009 dollars'])
    for i in range(len(GDP['GDP in billions of chained 2009 dollars'])):
        if GDP['GDP in billions of chained 2009 dollars'][i] > GDP['GDP in billions of chained 2009 dollars'][i+1]:
            if GDP['GDP in billions of chained 2009 dollars'][i + 1] > GDP['GDP in billions of chained 2009 dollars'][i+2]:
                value_rec = GDP['GDP in billions of chained 2009 dollars'][i+2]
                j = i
                while GDP['GDP in billions of chained 2009 dollars'][j] / GDP['GDP in billions of chained 2009 dollars'][j+1] < 0 or j < len(GDP['GDP in billions of chained 2009 dollars']):
                    if (GDP['GDP in billions of chained 2009 dollars'][j] < GDP['GDP in billions of chained 2009 dollars'][j+1]):
                        return GDP.iloc[j+2].name
#                     j = i
                    j += 1 
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows=220, header=None)
#     print(GDP)
    j = 0
    GDP = GDP[[4, 5, 6]]
    GDP = pd.DataFrame(GDP)
    GDP.rename(columns={4: 'date', 5 : 'GDP in billions of current dollars', 6 : 'GDP in billions of chained 2009 dollars'}, inplace=True)
    GDP.set_index('date', inplace=True)
#     print(GDP['GDP in billions of chained 2009 dollars'])
    for i in range(len(GDP['GDP in billions of chained 2009 dollars'])):
        if GDP['GDP in billions of chained 2009 dollars'][i] > GDP['GDP in billions of chained 2009 dollars'][i+1]:
            if GDP['GDP in billions of chained 2009 dollars'][i + 1] > GDP['GDP in billions of chained 2009 dollars'][i+2]:
                value_rec = GDP['GDP in billions of chained 2009 dollars'][i+2]
                j = i
                while GDP['GDP in billions of chained 2009 dollars'][j] > GDP['GDP in billions of chained 2009 dollars'][j+1] or j < len(GDP['GDP in billions of chained 2009 dollars']):
                    if (GDP['GDP in billions of chained 2009 dollars'][j] < GDP['GDP in billions of chained 2009 dollars'][j+1]):
                        return GDP.iloc[j].name
#                     j = i
                    j += 1 
get_recession_bottom()

In [ ]:
import re
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
#     print(house)
    for n in range(1997, 2000):
        for month in range(1, 13):
            if (month < 10):
                house.drop([str(n)+'-' + '0' + str(month)],axis=1,inplace=True)
            else:
                house.drop([str(n)+'-' + str(month)],axis=1,inplace=True)
    for month in range(4, 13):
        if (month < 10):
            house.drop(['1996-' + '0' + str(month)],axis=1,inplace=True)
        else:
            house.drop(['1996-' + str(month)],axis=1,inplace=True)
    for y in range(16):
        for m in range(12):
            if (y < 10):
                house.rename(columns={'200'+ str(y)+'-' + str(m) : '200'+ str(y)+'-' + str(m)}, inplace=True)
            else:
                house.rename(columns={'20'+ str(y)+'-' + str(m) : '20'+ str(y)+'-' + str(m)}, inplace=True)
    for i in range(16):
        for quart in range(1, 13, 3):
            if i < 10:
                if quart < 4:
                    house['200' + str(i) + 'q1'] = (house[['200' + str(i) + '-' + '0' + str(quart), '200' + str(i) + '-' + '0' + str(quart + 1), '200' + str(i) + '-' + '0' + str(quart + 2)]]).mean() 
                elif quart > 3 and quart < 7:
                    house['200' + str(i) + 'q2'] = (house['200' + str(i) + '-' + '0' + str(quart)] + house['200' + str(i) + '-' + '0' + str(quart + 1)] + house['200' + str(i) + '-' + '0' + str(quart + 2)]) / 3 
                elif quart > 6 and quart < 10:
                    house['200' + str(i) + 'q3'] = (house['200' + str(i) + '-' + '0' + str(quart)] + house['200' + str(i) + '-' + '0' + str(quart + 1)] + house['200' + str(i) + '-' + '0'+ str(quart + 2)]) / 3 
                else:
                    house['200' + str(i) + 'q4'] = (house['200' + str(i) + '-' + str(quart)] + house['200' + str(i) + '-' + str(quart + 1)] + house['200' + str(i) + '-' + str(quart + 2)]) / 3 
            else:
                if quart < 4:
                    house['20' + str(i) + 'q1'] = (house['20' + str(i) + '-' + '0' + str(quart)] + house['20' + str(i) + '-' + '0' + str(quart + 1)] + house['20' + str(i) + '-' + '0' + str(quart + 2)]) / 3 
                elif quart > 3 and quart < 7:
                    house['20' + str(i) + 'q2'] = (house['20' + str(i) + '-' + '0' + str(quart)] + house['20' + str(i) + '-' + '0' + str(quart + 1)] + house['20' + str(i) + '-' + '0' + str(quart + 2)]) / 3 
                elif quart > 6 and quart < 10:
                    house['20' + str(i) + 'q3'] = (house['20' + str(i) + '-' + '0' + str(quart)] + house['20' + str(i) + '-' + '0' + str(quart + 1)] + house['20' + str(i) + '-' + '0'+ str(quart + 2)]) / 3 
                else:
                    house['20' + str(i) + 'q4'] = (house['20' + str(i) + '-' + str(quart)] + house['20' + str(i) + '-' + str(quart + 1)] + house['20' + str(i) + '-' + str(quart + 2)]) / 3 
    
    house['2016q1'] = house[['2016-01', '2016-02', '2016-03']].mean(axis=0)
    house['2016q2'] = house[['2016-04', '2016-05', '2016-06']].mean(axis=0)
    house['2016q3'] = np.nan
    for year in range(2000, 2016):
        for m in range(1,13):
            if m > 9:
                house = house.drop(str(year) + '-' + str(m), axis=1)
            else:
                house = house.drop(str(year) + '-' + '0'+ str(m), axis=1)
    for m in range(1, 9):
        house = house.drop('2016-0' + str(m), axis=1)
    house = house.drop(['SizeRank', 'RegionID', 'Metro', 'CountyName'], axis=1)
    house['State'] = house['State'].replace(states)
#     print(states.values())
    house.set_index(['State', 'RegionName'], inplace=True)
    return house
convert_housing_data_to_quarters()
# convert_housing_data_to_quarters().loc["TX"].loc["Austin"].loc["2010q3"]

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    from scipy.stats import ttest_ind
    start = get_recession_start()
    bottom = get_recession_bottom()
    house = convert_housing_data_to_quarters()
    d_list = []
    for y in range(2008, 2010):
        for q in range(1, 5):
            if (y == 2008 and q > 2) or (y==2009 and q < 3):
                date = str(y) + 'q' + str(q)
                d_list.append(date)
    uni_t = get_list_of_university_towns()
    uni_t['U_T'] = 1
    uni_t.set_index(['State', 'RegionName'], inplace=True)
#     print(uni_t)
    house = house[d_list]
    house_price = house.merge(uni_t, how='left', left_index = True, right_index = True)
    house_price.U_T[house_price.U_T.isnull()] = 0
    scoup = house_price[[start, bottom, 'U_T']]
    scoup['change'] = scoup[bottom] / scoup[start]
    ut = scoup[scoup['U_T'] == 1]['change'].dropna()
    not_ut = scoup[scoup['U_T'] == 0]['change'].dropna()
    t = ttest_ind(ut, not_ut)
    p = t[1]
    return p < 0.01 and True or False, p, (ut.mean() < not_ut.mean()) and 'non-university town' or 'university town'
run_ttest()